In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor


df = pd.read_csv('Data/laptop_prices.csv')

df


,Brand,Processor,RAM (GB),Storage,GPU,Screen Size (inch),Resolution,Battery Life (hours),Weight (kg),Operating System,Price ($)
0,Apple,AMD Ryzen 3,64,512GB SSD,Nvidia GTX 1650,17.3,2560x1440,8.9,1.42,FreeDOS,3997.07
1,Razer,AMD Ryzen 7,4,1TB SSD,Nvidia RTX 3080,14.0,1366x768,9.4,2.57,Linux,1355.78
2,Asus,Intel i5,32,2TB SSD,Nvidia RTX 3060,13.3,3840x2160,8.5,1.74,FreeDOS,2673.07
3,Lenovo,Intel i5,4,256GB SSD,Nvidia RTX 3080,13.3,1366x768,10.5,3.10,Windows,751.17
4,Razer,Intel i3,4,256GB SSD,AMD Radeon RX 6600,16.0,3840x2160,5.7,3.38,Linux,2059.83
...,...,...,...,...,...,...,...,...,...,...,...
11763,Acer,Intel i3,4,2TB SSD,Nvidia RTX 2060,17.3,1366x768,11.5,1.58,macOS,704.82
11764,Asus,Intel i3,4,2TB SSD,AMD Radeon RX 6800,16.0,1366x768,9.5,2.14,Linux,775.59
11765,Razer,AMD Ryzen 9,4,2TB SSD,AMD Radeon RX 6600,15.6,2560x1440,8.2,2.05,Linux,2789.46
11766,Samsung,AMD Ryzen 7,16,512GB SSD,Integrated,13.3,1920x1080,7.5,1.48,macOS,1067.13


In [2]:
from sklearn.preprocessing import LabelEncoder

le_Brand = LabelEncoder()
le_Processor = LabelEncoder()
le_RAM = LabelEncoder()
le_Storage = LabelEncoder()
le_GPU = LabelEncoder()
le_OS = LabelEncoder()
le_Resolution = LabelEncoder()

df['Brand_en'] = le_Brand.fit_transform(df['Brand'])
df['Processor_en'] = le_Processor.fit_transform(df['Processor'])
df['RAM_en'] = le_RAM.fit_transform(df['RAM (GB)'])
df['Storage_en'] = le_Storage.fit_transform(df['Storage'])
df['GPU_en'] = le_GPU.fit_transform(df['GPU'])
df['OS_en'] = le_OS.fit_transform(df['Operating System'])
df['Resolution_en'] = le_Resolution.fit_transform(df['Resolution'])

print(dict(enumerate(le_Brand.classes_)))
print(dict(enumerate(le_Processor.classes_)))
print(dict(enumerate(le_RAM.classes_)))
print(dict(enumerate(le_Storage.classes_)))
print(dict(enumerate(le_GPU.classes_)))
print(dict(enumerate(le_OS.classes_)))
print(dict(enumerate(le_Resolution.classes_)))


{0: 'Acer', 1: 'Apple', 2: 'Asus', 3: 'Dell', 4: 'HP', 5: 'Lenovo', 6: 'MSI', 7: 'Microsoft', 8: 'Razer', 9: 'Samsung'}
{0: 'AMD Ryzen 3', 1: 'AMD Ryzen 5', 2: 'AMD Ryzen 7', 3: 'AMD Ryzen 9', 4: 'Intel i3', 5: 'Intel i5', 6: 'Intel i7', 7: 'Intel i9'}
{0: np.int64(4), 1: np.int64(8), 2: np.int64(16), 3: np.int64(32), 4: np.int64(64)}
{0: '1TB HDD', 1: '1TB SSD', 2: '256GB SSD', 3: '2TB SSD', 4: '512GB SSD'}
{0: 'AMD Radeon RX 6600', 1: 'AMD Radeon RX 6800', 2: 'Integrated', 3: 'Nvidia GTX 1650', 4: 'Nvidia RTX 2060', 5: 'Nvidia RTX 3060', 6: 'Nvidia RTX 3080'}
{0: 'FreeDOS', 1: 'Linux', 2: 'Windows', 3: 'macOS'}
{0: '1366x768', 1: '1920x1080', 2: '2560x1440', 3: '3840x2160'}


In [3]:
import re

def convert_to_gb(value):
    if "TB" in value:
        return int(re.search(r'\d+', value).group()) * 1024
    elif "GB" in value:
        return int(re.search(r'\d+', value).group())
    return 0

# สร้างคอลัมน์ใหม่
df["HDD (GB)"] = df["Storage"].apply(lambda x: convert_to_gb(x) if "HDD" in x else 0)
df["SSD (GB)"] = df["Storage"].apply(lambda x: convert_to_gb(x) if "SSD" in x else 0)

In [4]:
df = df.drop(['Brand', 'Processor', 'RAM (GB)', 'Storage', 'GPU', 'Operating System', 'Resolution', 'Storage_en'], axis=1)

df

,Screen Size (inch),Battery Life (hours),Weight (kg),Price ($),Brand_en,Processor_en,RAM_en,GPU_en,OS_en,Resolution_en,HDD (GB),SSD (GB)
0,17.3,8.9,1.42,3997.07,1,0,4,3,0,2,0,512
1,14.0,9.4,2.57,1355.78,8,2,0,6,1,0,0,1024
2,13.3,8.5,1.74,2673.07,2,5,3,5,0,3,0,2048
3,13.3,10.5,3.10,751.17,5,5,0,6,2,0,0,256
4,16.0,5.7,3.38,2059.83,8,4,0,0,1,3,0,256
...,...,...,...,...,...,...,...,...,...,...,...,...
11763,17.3,11.5,1.58,704.82,0,4,0,4,3,0,0,2048
11764,16.0,9.5,2.14,775.59,2,4,0,1,1,0,0,2048
11765,15.6,8.2,2.05,2789.46,8,3,0,0,1,2,0,2048
11766,13.3,7.5,1.48,1067.13,9,2,2,2,3,1,0,512


In [5]:
import numpy as np
import pandas as pd

def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df_cleaned = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

    return df_cleaned

# 🔹 ลบ Outliers เฉพาะคอลัมน์ Price ($)
df_cleaned = remove_outliers_iqr(df, 'Price ($)')

# 🔹 แสดงจำนวนแถวก่อนและหลังการลบ Outliers
original_size = df.shape[0]
cleaned_size = df_cleaned.shape[0]

original_size, cleaned_size


(11768, 11163)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler_y = StandardScaler()
scaler_all = StandardScaler()

x1 = df_cleaned['RAM_en']
x2 = df_cleaned['Resolution_en']
x3 = df_cleaned['Processor_en']
x4 = df_cleaned['Battery Life (hours)']
X = np.column_stack((x1, x2, x3, x4))
y = df_cleaned['Price ($)']
X_all = scaler_all.fit_transform(df)
X_all_df = pd.DataFrame(X_all, columns=df.columns)

X_scaled = scaler.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()


In [18]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# แปลงข้อมูลเป็น DataFrame
data = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3,'y': y})

# กำหนดค่าเริ่มต้นของพารามิเตอร์ m1, m2 และ c
m1, m2, m3, c = (400, 400, 100, 500)

# กำหนด learning rate และจำนวนรอบการปรับค่าพารามิเตอร์
learning_rate = 0.00001
epochs = 10000

# เก็บค่าของสมการในแต่ละ epoch
equations = []

# เริ่มต้น Gradient Descent
for epoch in range(epochs):
    # คำนวณค่าพยากรณ์ y_pred
    y_pred = m1*x1 + m2*x2 + m3*x3 + c
    
    # คำนวณค่าความผิดพลาด (Error)
    error = y - y_pred
    
    # คำนวณค่า Gradient Descent
    dm1 = -2 * np.sum(x1 * error) / len(y)
    dm2 = -2 * np.sum(x2 * error) / len(y)
    dm3 = -2 * np.sum(x3 * error) / len(y)
    dc = -2 * np.sum(error) / len(y)
    
    # อัปเดตค่าพารามิเตอร์
    m1 -= learning_rate * dm1
    m2 -= learning_rate * dm2
    m3 -= learning_rate * dm3
    c -= learning_rate * dc

    # MAE MSE
    mae = mean_absolute_error(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    
    # บันทึกสมการที่อัปเดต
    if epoch % 100 == 0:  # บันทึกทุก ๆ 100 รอบ
        equations.append(f"Epoch {epoch}: y = {m1:.2f}x1 + {m2:.2f}x2 + {m3:.2f}x3 + {c:.2f} ----- MAE: {mae:.2f} -- MSE: {mse:.2f}")

# แสดงสมการที่ได้
equations[10:]  # แสดงสมการที่บันทึกไว้ใน 10 รอบแรก


['Epoch 1000: y = 393.72x1 + 394.19x2 + 87.14x3 + 496.40 ----- MAE: 525.13 -- MSE: 425108.24',
 'Epoch 1100: y = 393.24x1 + 393.71x2 + 86.16x3 + 496.11 ----- MAE: 523.83 -- MSE: 423600.52',
 'Epoch 1200: y = 392.78x1 + 393.26x2 + 85.22x3 + 495.83 ----- MAE: 522.60 -- MSE: 422221.72',
 'Epoch 1300: y = 392.33x1 + 392.81x2 + 84.33x3 + 495.56 ----- MAE: 521.46 -- MSE: 420960.66',
 'Epoch 1400: y = 391.91x1 + 392.38x2 + 83.48x3 + 495.30 ----- MAE: 520.39 -- MSE: 419807.17',
 'Epoch 1500: y = 391.51x1 + 391.97x2 + 82.67x3 + 495.05 ----- MAE: 519.39 -- MSE: 418751.93',
 'Epoch 1600: y = 391.12x1 + 391.57x2 + 81.90x3 + 494.80 ----- MAE: 518.45 -- MSE: 417786.42',
 'Epoch 1700: y = 390.75x1 + 391.18x2 + 81.17x3 + 494.57 ----- MAE: 517.57 -- MSE: 416902.90',
 'Epoch 1800: y = 390.40x1 + 390.80x2 + 80.47x3 + 494.34 ----- MAE: 516.76 -- MSE: 416094.25',
 'Epoch 1900: y = 390.06x1 + 390.44x2 + 79.80x3 + 494.12 ----- MAE: 515.99 -- MSE: 415354.01',
 'Epoch 2000: y = 389.74x1 + 390.08x2 + 79.16x3 + 